In [ ]:
GetOldTweets3 -h

# GetOldTweet3 패키지 설치

In [ ]:
pip install GetOldTweets3

In [11]:
import GetOldTweets3 as got

# BeautifulSoup4 설치 확인

In [12]:
try:
    from bs4 import BeautifulSoup
except:
    !pip install bs4
    from bs4 import beautifulSoup

# 수집기간 정의

In [13]:
# 가져올 범위 정의
# 트럼프 임기 시작일: 2017년 1월 20일

import datetime    # datetime 패키지

days_range = []


# https://datascienceschool.net/view-notebook/465066ac92ef4da3b0aba32f76d9750a/
# datetime 패키지, datetime 클래스(날짜와 시간 함께 저장)
# strptime 메서드: 문자열 반환
start = datetime.datetime.strptime("2017-01-20", "%Y-%m-%d")
end = datetime.datetime.strptime("2019-12-31", "%Y-%m-%d")

date_generated = [start+datetime.timedelta(days=x) for x in range(0, (end-start).days)]

for date in date_generated:
    days_range.append(date.strftime("%Y-%m-%d"))
                                    
print("===설정된 트윗 수집 기간: {} ~ {}===".format(days_range[0], days_range[-1]))
print("===총 {}일 간의 데이터 수집===".format(len(days_range)))

===설정된 트윗 수집 기간: 2017-01-20 ~ 2019-12-30===
===총 1075일 간의 데이터 수집===


# 트윗 수집하기

In [26]:
# tweetCriteria로 수집 기준 정의
tweetCriteria = got.manager.TweetCriteria().setUsername("realDonaldTrump").setSince("2017-01-20").setUntil("2019-12-31")

In [27]:
import time

# 수집
print("Collecting data start.. from {} to {}".format(days_range[0], days_range[-1]))
start_time = time.time()

tweet = got.manager.TweetManager.getTweets(tweetCriteria)

print("Collecting data end.. {0:0.2f} Minutes".format((time.time() - start_time)/60))
print("=== Total num of tweets is {} ===".format(len(tweet)))

=== Total num of tweets is 9670 ===


# 변수 저장하기

In [30]:
# 원하는 정보만 저장
# 유저 아이디, 트윗 내용, 날짜, 좋아요 수, 리트윗 수, 링크

from tqdm import tqdm_notebook

# initialize
tweet_list = []

for i in tqdm_notebook(tweet):
    
    username = i.username
    tweet_date = i.date.strftime("%Y-%m-%d")
    tweet_time = i.date.strftime("%H:%M:%dS")
    content = i.text
    favorites = i.favorites
    retweets = i.retweets
    link = i.permalink
    
    info_list = [username, tweet_date, tweet_time, content, favorites, retweets, link]
    tweet_list.append(info_list)

# 파일 저장하기

In [31]:
import pandas as pd

# 데이터프레임 생성
twitter_df = pd.DataFrame(tweet_list, columns = ["ID", "날짜", "시간", "내용", "리트윗 수", "좋아요 수", "링크"])


# csv 파일 만들기
twitter_df.to_csv("sample_twitter_data_{}_to{}.csv".format(days_range[0], days_range[-1]), index = False)
print("=== {} tweets are successfully saved ===".format(len(tweet_list)))

=== 9670 tweets are successfully saved ===


# 파일 확인하기

In [49]:
df_tweet = pd.read_csv("sample_twitter_data_{}_to{}.csv".format(days_range[0], days_range[-1]))
df_tweet.head(10)

,ID,날짜,시간,내용,리트윗 수,좋아요 수,링크
0,realDonaldTrump,2019-12-30,22:23:30S,"He is Fake News, will always be Fredo to us. I...",79417,19153,https://twitter.com/realDonaldTrump/status/121...
1,realDonaldTrump,2019-12-30,14:28:30S,Thank you to highly respected Jewish leader Do...,110202,20738,https://twitter.com/realDonaldTrump/status/121...
2,realDonaldTrump,2019-12-29,19:10:29S,"The anti-Semitic attack in Monsey, New York, o...",160068,34627,https://twitter.com/realDonaldTrump/status/121...
3,realDonaldTrump,2019-12-29,14:55:29S,Crazy Nancy Pelosi should spend more time in h...,123696,37021,https://twitter.com/realDonaldTrump/status/121...
4,realDonaldTrump,2019-12-29,12:57:29S,Any answers Nancy?,112552,29914,https://twitter.com/realDonaldTrump/status/121...
5,realDonaldTrump,2019-12-29,00:00:29S,Tremendous Spirit!,92777,22814,https://twitter.com/realDonaldTrump/status/121...
6,realDonaldTrump,2019-12-28,23:58:28S,Thank you Joe. So good for our workers!,56431,13434,https://twitter.com/realDonaldTrump/status/121...
7,realDonaldTrump,2019-12-28,22:15:28S,A Dirty Cop at the highest level. Scum!,67283,19403,https://twitter.com/realDonaldTrump/status/121...
8,realDonaldTrump,2019-12-28,22:13:28S,Thank you Adriana!,33357,7508,https://twitter.com/realDonaldTrump/status/121...
9,realDonaldTrump,2019-12-28,21:14:28S,So sad to see that New York City and State are...,137563,29538,https://twitter.com/realDonaldTrump/status/121...


In [52]:
df_tweet['내용'].to_csv("trump")

C:\Users\Playdata\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [ ]:
자연어 처리
- 의미없는 동사(be, 조동사 등등) 제외
- 특수기호 제외
- 

In [ ]:
다른 대통령 트위터와 비교
빈도와 주가 상관관계

In [59]:
crazy_date = df_tweet['날짜'] == '2019-03-17'

In [67]:
crazy_day = df_tweet[crazy_date]
crazy_day

,ID,날짜,시간,내용,리트윗 수,좋아요 수,링크
3928,realDonaldTrump,2019-03-17,23:16:17S,What the Democrats have done in trying to stea...,117913,31997,https://twitter.com/realDonaldTrump/status/110...
3929,realDonaldTrump,2019-03-17,22:27:17S,"Just spoke to Mary Barra, CEO of General Motor...",104474,21342,https://twitter.com/realDonaldTrump/status/110...
3930,realDonaldTrump,2019-03-17,20:58:17S,Those Republican Senators who voted in favor o...,85166,19098,https://twitter.com/realDonaldTrump/status/110...
3931,realDonaldTrump,2019-03-17,18:18:17S,"Were @FoxNews weekend anchors, @ArthelNeville ...",73649,18653,https://twitter.com/realDonaldTrump/status/110...
3932,realDonaldTrump,2019-03-17,17:38:17S,Democrat UAW Local 1112 President David Green ...,59097,13355,https://twitter.com/realDonaldTrump/status/110...
3933,realDonaldTrump,2019-03-17,13:53:17S,NaN,44036,12079,https://twitter.com/realDonaldTrump/status/110...
3934,realDonaldTrump,2019-03-17,13:44:17S,....to the people that got you there. Keep fig...,104621,28649,https://twitter.com/realDonaldTrump/status/110...
3935,realDonaldTrump,2019-03-17,13:33:17S,....must stay strong and fight back with vigor...,96276,23301,https://twitter.com/realDonaldTrump/status/110...
3936,realDonaldTrump,2019-03-17,13:18:17S,Bring back @JudgeJeanine Pirro. The Radical Le...,100225,30914,https://twitter.com/realDonaldTrump/status/110...
3937,realDonaldTrump,2019-03-17,12:50:17S,Happy St. Patrick’s Day!,64096,11950,https://twitter.com/realDonaldTrump/status/110...
